# Decorators

Decorators are a Python-specific way to modify or enhance existing functions. They're just Pythonic syntactic sugar for higher-order functions (and sometimes closures).

A function can be used as a decorator if it takes a function as an argument and returns a new function. The new function can then be used in place of the original function. Again, this is typically used to modify or enhance the behavior of the original function.

In [ ]:
def file_type_aggregator(func_to_wrap):
    # dict of file_type -> count
    counts = {}

    def wrapper(doc, file_type):
        nonlocal counts
        if file_type not in counts.keys():
            counts.update({file_type: 1})
        else:
            counts[file_type] += 1
        result = func_to_wrap(doc, file_type)

        return result, counts

    return wrapper


@file_type_aggregator
def process_doc(doc, file_type):
    return f"Processing doc: {doc} with File Type: {file_type}"


""" COURSE SOLUTION
def file_type_aggregator(func_to_wrap):
    # dict of file_type -> count
    counts = {}

    def wrapper(doc, file_type):
        nonlocal counts

        if file_type not in counts:
            counts[file_type] = 0
        counts[file_type] += 1
        result = func_to_wrap(doc, file_type)

        return result, counts

    return wrapper


@file_type_aggregator
def process_doc(doc, file_type):
    return f"Processing doc: {doc} with File Type: {file_type}"
"""


In [ ]:
# TESTS

run_cases = [
    (
        ("Welcome to the jungle", "txt"),
        ("Processing doc: Welcome to the jungle with File Type: txt", {"txt": 1}),
    ),
    (
        ("We've got fun and games", "txt"),
        ("Processing doc: We've got fun and games with File Type: txt", {"txt": 2}),
    ),
    (
        ("We've got *everything* you want honey", "md"),
        (
            "Processing doc: We've got *everything* you want honey with File Type: md",
            {"txt": 2, "md": 1},
        ),
    ),
]

submit_cases = run_cases + [
    (
        ("We are the champions my friends", "docx"),
        (
            "Processing doc: We are the champions my friends with File Type: docx",
            {"txt": 2, "md": 1, "docx": 1},
        ),
    ),
    (
        ("print('hello world')", "py"),
        (
            "Processing doc: print('hello world') with File Type: py",
            {"txt": 2, "md": 1, "docx": 1, "py": 1},
        ),
    ),
]


def test(inputs, expected_output):
    print("---------------------------------")
    print(f"Inputs: {inputs}")
    print(f"Expecting: {expected_output}")
    counts = process_doc(*inputs)
    print(f"Actual: {counts}")
    if counts == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()


# Args and kwargs

In Python, *args and **kwargs are Pythonic syntax that allows a function to accept and deal with a variable number of arguments in a function.

    *args collects positional arguments into a tuple
    **kwargs collects keyword arguments into a dictionary

## A note on ordering

Any positional arguments must come before keyword arguments. Python will try to match up the arguments you pass in with the arguments in the function definition by position first, and then by name.

In [ ]:
def args_logger(*args, **kwargs):
    for arg in args:
        print(f"* {arg}")
    for key, value in sorted(kwargs.items()):
        print(f"* {key}: {value}")

In [ ]:
# Don't edit below this line


def test(*args, **kwargs):
    args_logger(*args, **kwargs)
    print("========================================")


def main():
    test(1, 2, date_str="01/01/2023")
    test(message="Hello World", to_delete="l")
    test(1, 2, 3, 4, 5)
    test("hi", True, name="Lane", age=28)


main()


In [ ]:
def markdown_to_text_decorator(func):
    def wrapper(*args, **kwargs):
        converted_args = []
        converted_kwargs = {}
        for arg in args:
            converted_args.append(convert_md_to_txt(arg))
        for key, value in kwargs.items():
            converted_kwargs[key] = convert_md_to_txt(value)
        return func(*converted_args, **converted_kwargs)
    return wrapper


def convert_md_to_txt(doc):
    processed_lines = []
    for line in doc.split("\n"):
        processed_lines.append(line.lstrip('#').strip())
    return "\n".join(processed_lines)


# Don't edit below this line


@markdown_to_text_decorator
def concat(first_doc, second_doc):
    return f"""First: {first_doc}
Second: {second_doc}
"""


@markdown_to_text_decorator
def format_as_essay(title, body, conclusion):
    return f"""Title: {title}
Body: {body}
Conclusion: {conclusion}
"""


In [ ]:
# TESTS

run_cases = [
    (
        ("# We like to play it all", "## Welcome to Tally Hall"),
        {},
        concat,
        "First: We like to play it all\nSecond: Welcome to Tally Hall\n",
    ),
    (
        set(),
        {
            "conclusion": "## That's why Python is great!",
            "title": "Why Python is Great",
            "body": """Python is a great language.

## Reasons:
* It's really easy to learn
* Everyone uses it
""",
        },
        format_as_essay,
        """Title: Why Python is Great
Body: Python is a great language.

Reasons:
* It's really easy to learn
* Everyone uses it

Conclusion: That's why Python is great!
""",
    ),
]

submit_cases = run_cases + [
    (
        ("She's a killer queen",),
        {
            "second_doc": "### Gunpowder, gelatine",
        },
        concat,
        "First: She's a killer queen\nSecond: Gunpowder, gelatine\n",
    ),
    (
        (
            "Boots' grocery list",
            """Don't forget!

## Grocery List:
* Salmon
* Honey
* Quest Scrolls
""",
        ),
        {
            "conclusion": "## Boots is getting ready for the weekend!",
        },
        format_as_essay,
        """Title: Boots' grocery list
Body: Don't forget!

Grocery List:
* Salmon
* Honey
* Quest Scrolls

Conclusion: Boots is getting ready for the weekend!
""",
    ),
]


def test(args, kwargs, func, expected_output):
    print("---------------------------------")
    print(f"\nVariadic Arguments:\n{args}\n")
    print(f"Keyword Arguments:\n{kwargs}\n")
    print(f"Expecting:\n{expected_output}")
    try:
        result = func(*args, **kwargs)
    except Exception as error:
        result = f"Error: {error}"
    print(f"Actual:\n{result}")
    if result == expected_output:
        print("Pass")
        return True
    print("Fail")
    return False


def main():
    passed = 0
    failed = 0
    for test_case in test_cases:
        correct = test(*test_case)
        if correct:
            passed += 1
        else:
            failed += 1
    if failed == 0:
        print("============= PASS ==============")
    else:
        print("============= FAIL ==============")
    print(f"{passed} passed, {failed} failed")


test_cases = submit_cases
if "__RUN__" in globals():
    test_cases = run_cases

main()
